# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [106]:
#Loading enviroment viariables
%load_ext dotenv
%dotenv 

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [58]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [60]:
import os
from glob import glob
import pandas as pd

#Loading the 'PRICE_DATA' variable  
PRICE_DATA = os.getenv("PRICE_DATA")

#Finding the path of all parquet files in the directory PRiCE_DATA
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [61]:
#Adding parquet files to dd dataframe
dd_px = dd.read_parquet(parquet_files)
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463043,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142937,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652859,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219185,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237469,2819626,Health Care,Life Sciences Tools & Services,2000


In [107]:
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [115]:
df_feat = dd_feat.compute()

In [116]:
df_feat['rolling_average'] = df_feat['returns'].rolling(10).mean()

In [120]:
df_feat.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,rolling_average
ticker,,,,,,,,,,,,,,
HUM,2010-01-04,44.430000,45.770000,44.430000,45.680000,40.787460,2008500,Health Care,Managed Health Care,2010,43.889999,0.040784,1.340000,NaN
HUM,2010-01-05,45.790001,46.160000,45.270000,45.660000,40.769611,1860700,Health Care,Managed Health Care,2010,45.680000,-0.000438,0.889999,NaN
HUM,2010-01-06,45.660000,46.340000,45.349998,45.980000,41.055332,1686800,Health Care,Managed Health Care,2010,45.660000,0.007008,0.990002,NaN
HUM,2010-01-07,46.250000,48.689999,46.250000,48.119999,42.966137,3706600,Health Care,Managed Health Care,2010,45.980000,0.046542,2.439999,NaN
HUM,2010-01-08,47.810001,48.020000,47.200001,47.509998,42.421463,2412300,Health Care,Managed Health Care,2010,48.119999,-0.012677,0.820000,NaN
HUM,2010-01-11,47.480000,47.730000,46.709999,46.950001,41.921440,2000900,Health Care,Managed Health Care,2010,47.509998,-0.011787,1.020000,NaN
HUM,2010-01-12,46.480000,46.779999,45.619999,46.180000,41.233913,2392100,Health Care,Managed Health Care,2010,46.950001,-0.016400,1.160000,NaN
HUM,2010-01-13,46.240002,47.139999,45.320000,47.029999,41.992874,2455800,Health Care,Managed Health Care,2010,46.180000,0.018406,1.820000,NaN
HUM,2010-01-14,46.759998,48.599998,46.750000,48.020000,42.876858,3433400,Health Care,Managed Health Care,2010,47.029999,0.021050,1.849998,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [119]:
#It was not necessary to convert to pandas to calculate the moving average return as that operation can be completed
#similarly to how 'returns' and 'Hi_Low_Range' were calculated. See demonstration below. It is faster to run in dask as
#converting to pandas took approximately 2 and a half minutes. 

#Calculating the rolling average without converting
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(rolling_average = x['returns'].rolling(10).mean())
))

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.